In [6]:
# path='dataset/video/urg3.mp4'
# out_path='dataset/image/urg3'
# cap=cv2.VideoCapture(path)
# frame=0
# while True:
#     ret,image=cap.read()
#     if not ret:
#         break
#     image=cv2.resize(image,(300,300))
#     cv2.imwrite(os.path.join(out_path,'img_%s'%(frame)+'.jpg'),image)
#     frame+=1
# cap.release()    

In [19]:
import onnxruntime
import onnx
import json
import numpy as np
import cv2, os

In [20]:
session=onnxruntime.InferenceSession('models/mb1-ssd-BDD-all-Epoch-199-Loss-3.2500881144874976kkk.onnx',providers=['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider'])
input_name=session.get_inputs()[0].name
output_name1=session.get_outputs()[0].name
output_name2=session.get_outputs()[1].name

2022-12-15 08:44:16.378909366 [W:onnxruntime:Default, onnxruntime_pybind_state.cc:509 CreateExecutionProviderInstance] Failed to create TensorrtExecutionProvider. Please reference https://onnxruntime.ai/docs/execution-providers/TensorRT-ExecutionProvider.html#requirements to ensure all dependencies are met.


In [22]:
img_list=os.listdir("images/demo_ces")
# out1=cv2.VideoWriter("results/foggy.mp4",cv2.VideoWriter_fourcc(*'XVID'),10.0,(300, 300))
# out2=cv2.VideoWriter("results/foggy1.mp4",cv2.VideoWriter_fourcc(*'XVID'),10.0,(3840, 2160))
conf_thres=0

for img in img_list:
    orig_image=cv2.imread(os.path.join("images/demo_ces",img))
    h,w,_=orig_image.shape
    image=cv2.cvtColor(orig_image,cv2.COLOR_BGR2RGB)
    image=cv2.resize(image,(300,300))
    image=image.reshape(1,3,300,300)
    image=np.array(image,dtype=np.float32)/255
    result=session.run([output_name1,output_name2],{input_name:image})
    for i in range(3000):
        conf=max(result[0][0][i])
        if conf>conf_thres:
            box=result[1][0][i]
            box=np.array(box*300,dtype=np.uint8)
            cv2.rectangle(orig_image, (box[0], box[1]), (box[2], box[3]), (0,255,0), 1)
            cv2.putText(orig_image, 'label: ' + str(np.argmax(result[0][0][i]))+', '+'conf: '+str(conf),
                        (box[0] - 10, box[1] - 10),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        .25,  # font scale
                        (255,0,0),
                        1)
        cv2.imwrite(os.path.join("images/onnx_ces",img),cv2.resize(orig_image,(w,h)))
    # out1.write(orig_image)
    # out2.write(cv2.resize(orig_image,(3840, 2160)))
    
# out1.release()
# out2.release()

KeyboardInterrupt: 

In [49]:
# with open("../dataset/ces_v1.json",'r') as file:
#     k=json.load(file)
    
# with open("../dataset/instances_default.json",'r') as file:
#     d=json.load(file)   

In [18]:
len(os.listdir("images/demo_ces"))

1080

In [24]:
!unzip "images/SampleFrames_for_Baselineline.zip"

Archive:  images/SampleFrames_for_Baselineline.zip
  inflating: SampleFrames_for_Baselineline/frame00434.jpg  
  inflating: SampleFrames_for_Baselineline/frame00435.jpg  
  inflating: SampleFrames_for_Baselineline/frame00436.jpg  
  inflating: SampleFrames_for_Baselineline/frame00437.jpg  
  inflating: SampleFrames_for_Baselineline/frame00438.jpg  
  inflating: SampleFrames_for_Baselineline/frame00439.jpg  
  inflating: SampleFrames_for_Baselineline/frame00440.jpg  
  inflating: SampleFrames_for_Baselineline/frame00441.jpg  
  inflating: SampleFrames_for_Baselineline/frame00442.jpg  
  inflating: SampleFrames_for_Baselineline/frame00443.jpg  
  inflating: SampleFrames_for_Baselineline/frame00444.jpg  
  inflating: SampleFrames_for_Baselineline/frame00445.jpg  
  inflating: SampleFrames_for_Baselineline/frame00446.jpg  
  inflating: SampleFrames_for_Baselineline/frame00447.jpg  
  inflating: SampleFrames_for_Baselineline/frame00448.jpg  
  inflating: SampleFrames_for_Baselineline/frame0

In [175]:
img='../../nandan/Experiments/CES/car_datasets/day_update/00054602-3bf57337.jpg'
orig_image=cv2.imread(img)
orig_image=cv2.resize(orig_image,(300,300))
image=cv2.cvtColor(orig_image,cv2.COLOR_BGR2RGB)
image=image.reshape(1,3,300,300)
image=np.array(image,dtype=np.float32)/255
result=session.run([output_name1,output_name2],{input_name:image})
# for i in range(3000):
#     conf=max(result[0][0][i])
#     if conf>conf_thres:
#         box=result[1][0][i]
#         box=np.array(box*300,dtype=np.uint8)
        
new_result=result[0][0][:,1:]
conf=[]
label=[]
for i in range(3000):
    conf.append(max(new_result[i]))
    label.append(np.argmax(new_result[i]))

In [176]:
max(conf),conf.index(max(conf)),label[conf.index(max(conf))]

(0.13718973, 2941, 1)

In [177]:
new_result[conf.index(max(conf))]

array([5.8403471e-04, 1.3718973e-01, 7.8167977e-05, 1.6515896e-03,
       8.2589562e-05, 1.7848126e-04, 5.9959702e-02, 6.7039909e-05,
       1.7642073e-03, 9.8146113e-05], dtype=float32)

In [178]:
[x for x in conf if x>0.1]

[0.12565388, 0.13718973, 0.107024, 0.117163405, 0.112683475]

In [180]:
with open("../dataset/instances_default.json",'r') as f:
    data=json.load(f)

In [14]:
from tensorflow.python import saved_model
import tensorflow as tf

In [17]:
# model=saved_model.builder.SavedModelBuilder('models/output_path.pb')
# model=tf.saved_model.load("models/frozen_inference_graph_coco.pb")
# model=tf.keras.models.load_model("models/frozen_inference_graph_coco.pb")

# model=tf.keras.models.load_model("../../deepak/car_d&n/pytorch-ssd/ver1pb")

In [10]:
# !zip images/base_imp.zip -r images/baseline_images images/improved_model

In [ ]:
# !apt-get install zip